In [ ]:
import numpy as np
import xarray as xr

import MDAnalysis as mda
from MDAnalysis.analysis.base import AnalysisBase
from MDAnalysis.lib.distances import distance_array

In [ ]:
triUb_CG_keywords = {'Protein': 'all',
                     'ProteinCA': 'name BB',
                     'Chain1': 'resid 1-76',
                     'Chain2': 'resid 77-152',
                     'Chain3': 'resid 153-228',
                     'Core1': 'resid 1-72',
                     'Core2': 'resid 77-148',
                     'Core3': 'resid 153-224',
                     'Chain1CA': 'resid 1-76 and name BB',
                     'Chain2CA': 'resid 77-152 and name BB',
                     'Chain3CA': 'resid 153-228 and name BB',
                     'Core1CA': 'resid 1-72 and name BB',
                     'Core2CA': 'resid 77-148 and name BB',
                     'Core3CA': 'resid 153-224 and name BB'}

In [ ]:
triUb_AA_keywords = {'Protein': 'all',
                     'ProteinCA': 'backbone',
                     'Chain1': 'resid 1-76',
                     'Chain2': 'resid 77-152',
                     'Chain3': 'resid 153-228',
                     'Core1': 'resid 1-72',
                     'Core2': 'resid 77-148', 
                     'Core3': 'resid 153-224',
                     'Chain1CA': 'resid 1-76 and name CA',
                     'Chain2CA': 'resid 77-152 and name CA',
                     'Chain3CA': 'resid 153-228 and name CA',
                     'Core1CA': 'resid 1-72 and name CA',
                     'Core2CA': 'resid 77-148 and name CA',
                     'Core3CA': 'resid 153-224 and name CA'}

In [ ]:
class calculate_RMD(AnalysisBase):
    """
    Calculates RMD values between each pair of atoms in two given groups.

    Parameters
    ----------
    atomgroup1 : MDAnalysis.core.groups.AtomGroup
    atomgroup2 : MDAnalysis.core.groups.AtomGroup

    Returns
    -------
    self : AnalysisBase
        MDAnalysis AnalysisBase object to run calculation
    """

    def __init__(self, atomgroup1, atomgroup2, backend="serial", **kwargs):
        super(calculate_RMD, self).__init__(atomgroup1.universe.trajectory, **kwargs)
        self._ag1 = atomgroup1
        self._ag2 = atomgroup2

        self.backend = backend

    def _prepare(self):
        self.mindist = np.empty((self.n_frames, self._ag1.n_atoms + self._ag2.n_atoms))
        self.times = np.empty((self.n_frames))

    def _single_frame(self):
        d = distance_array(self._ag1.positions, self._ag2.positions,
                           box=self._ts.dimensions,
                           backend=self.backend)

        self.mindist[self._frame_index] = np.hstack([d.min(axis=1), d.min(axis=0)]) / 10.0  # distance in nm
        self.times[self._frame_index] = self._ts.time  # time in ps

    def _conclude(self):
        resids = np.hstack([self._ag1.atoms.resids, self._ag2.atoms.resids])
        da = xr.DataArray(self.mindist,
                          dims=['time', 'resids'],
                          coords={'time': self.times, 'resids': resids},
                          attrs={"udata": "nm", "utime": "ps"},
                          name="minimum distance")

        if len(np.unique(resids)) != len(resids):
            # check if there is more than one atom per residue
            # and group them residue wise
            # without check this grouping would give a ValueError
            da = da.groupby("resids").min(dim="resids")

        self.result = da

## Compute RMD

In [ ]:
conformation = '/home/simonh/phd/BMBS_encodermap/upload_to_github/tri-ub_K48_CG.pdb'
trajectory = '/home/simonh/phd/BMBS_encodermap/upload_to_github/tri-ub_K48_minimal_example.xtc'
   
# create data output
output_file = '/home/simonh/phd/BMBS_encodermap/upload_to_github/tri-ub_K48_RMD_minimal_example.npy'

# choose resolution of given data
group_keywords = triUb_CG_keywords
# group_keywords = triUb_AA_keywords

# load simulation data
my_universe = mda.Universe(conformation, trajectory)

# run calculations for Core1 - Core2 dists
group1 = ("Core1CA", group_keywords["Core1CA"])
group2 = ("Core2CA", group_keywords["Core2CA"])

atomgroup1 = my_universe.select_atoms(group1[1])
atomgroup2 = my_universe.select_atoms(group2[1])
analysisbase = calculate_RMD(atomgroup1, atomgroup2)
analysisbase.run()
RWMD_12 = analysisbase.result.to_numpy()

# run calculations for Core2 - Core3 dists  
group1 = ("Core2CA", group_keywords["Core2CA"])
group2 = ("Core3CA", group_keywords["Core3CA"])

atomgroup1 = my_universe.select_atoms(group1[1])
atomgroup2 = my_universe.select_atoms(group2[1])
analysisbase = calculate_RMD(atomgroup1, atomgroup2)
analysisbase.run()
RWMD_23 = analysisbase.result.to_numpy()

# run calculations for Core1 - Core3 dists 
group1 = ("Core1CA", group_keywords["Core1CA"])
group2 = ("Core3CA", group_keywords["Core3CA"])

atomgroup1 = my_universe.select_atoms(group1[1])
atomgroup2 = my_universe.select_atoms(group2[1])
analysisbase = calculate_RMD(atomgroup1, atomgroup2)
analysisbase.run()
RWMD_13 = analysisbase.result.to_numpy()

# combined 1-2, 2-3 and 1-3 distances
RWMD_12_23_13 = np.hstack((RWMD_12, RWMD_23, RWMD_13))

# save hstacked output    
np.save(output_file, RWMD_12_23_13)

In [ ]:
np.shape(RWMD_12_23_13)